In [1]:
import pandas as pd

account=pd.read_parquet('account.gzip') 
card_dev=pd.read_parquet('card_dev.gzip') 
client=pd.read_parquet('client.gzip') 
disp=pd.read_parquet('disp.gzip') 
district=pd.read_parquet('district.gzip') 
loan_dev=pd.read_parquet('loan_dev.gzip') 
trans_dev=pd.read_parquet('trans_dev.gzip')

## Conclusões sobre chaves primárias

In [2]:
#tanto o disp_id como o client_id servem para chave primária
#(ou seja disp_id também funciona como o código do cliente)
disp.client_id.unique().shape==disp.disp_id.unique().shape

True

In [3]:
#não esquecer que existem contas com mais de um cliente
disp.account_id.unique().shape

(4500,)

In [4]:
#cada conta tem no máximo 2 pessoas
gb_ac=disp.groupby('account_id')['client_id'].count().reset_index()
gb_ac.loc[gb_ac.client_id>2]

,account_id,client_id


In [5]:
#tanto o disp_id como o card_id servem para chave primária
#(como o disp_id também funciona como o código do cliente, cada cliente tem no máx um cartão)
#(esta tabela só tem 1º tit!!)
card_dev.card_id.unique().shape==card_dev.disp_id.unique().shape

True

#### concluo que cada cliente tem no máx 1 cartão associado!

In [6]:
#tanto o account_id como o loan_id servem para chave primária
loan_dev.loan_id.unique().shape==loan_dev.account_id.unique().shape

True

### Perceber categorias na tabela trans

In [15]:
trans_dev['generation']= trans_dev['trans_date'].dt.strftime('%Y-%m')

In [16]:
trans_dev.loc[(trans_dev.account_id==61) & (trans_dev.operation.str.contains('remitt', na=False))].head()

,trans_id,account_id,type,operation,amount,balance,k_symbol,bank,account,trans_date,generation
73782,18045,61,withdrawal,remittance to another bank,1645.0,19150.0,household,EF,20533621.0,1994-05-13,1994-05
77744,18118,61,withdrawal,remittance to another bank,668.0,18516.3,Missing,IJ,20426410.0,1994-06-05,1994-06
80116,18046,61,withdrawal,remittance to another bank,1645.0,20890.3,household,EF,20533621.0,1994-06-13,1994-06
84325,18119,61,withdrawal,remittance to another bank,668.0,20294.7,Missing,IJ,20426410.0,1994-07-05,1994-07
86505,18047,61,withdrawal,remittance to another bank,1645.0,22668.7,household,EF,20533621.0,1994-07-13,1994-07


In [17]:
trans_dev.k_symbol.unique()

array(['Missing', 'interest credited', 'household',
       'payment for statement', 'insurrance payment',
       'sanction interest if negative balance', 'old-age pension'],
      dtype=object)

In [18]:
#sanction aparece quando o cartão teve uma penalização!
trans_dev.loc[trans_dev.k_symbol.str.contains('sanction', na=False)].shape

(305, 11)

In [19]:
trans_dev.loc[(trans_dev.balance<0) & (trans_dev.k_symbol.str.contains('sanction', na=False))].shape

(105, 11)

In [20]:
trans_dev.loc[(trans_dev.balance<0)].shape

(512, 11)

In [21]:
#contas arriscadas??
trans_dev.loc[(trans_dev.balance<0) | (trans_dev.k_symbol.str.contains('sanction', na=False))].shape

(712, 11)

In [22]:
#operation: collection from another bank - é o ordenado?
#k_symbol: interest credited - juros de uma conta poupança?
trans_dev.loc[trans_dev.account_id==5270].head(10)

,trans_id,account_id,type,operation,amount,balance,k_symbol,bank,account,trans_date,generation
0,1548749,5270,credit,credit in cash,800.0,800.0,Missing,Missing,NaN,1993-01-13,1993-01
1,1548750,5270,credit,collection from another bank,44749.0,45549.0,Missing,IJ,80269753.0,1993-01-14,1993-01
12,3669814,5270,credit,Missing,110.2,45659.2,interest credited,Missing,NaN,1993-01-31,1993-01
27,1549098,5270,withdrawal,withdrawal in cash,9600.0,36059.2,Missing,Missing,NaN,1993-02-12,1993-02
32,1548751,5270,credit,collection from another bank,44749.0,80808.2,Missing,IJ,80269753.0,1993-02-14,1993-02
55,3669815,5270,credit,Missing,268.5,81076.7,interest credited,Missing,NaN,1993-02-28,1993-02
89,1548752,5270,credit,collection from another bank,44749.0,125825.7,Missing,IJ,80269753.0,1993-03-14,1993-03
90,1549099,5270,withdrawal,withdrawal in cash,51700.0,74125.7,Missing,Missing,NaN,1993-03-14,1993-03
91,1549100,5270,withdrawal,withdrawal in cash,25300.0,48825.7,Missing,Missing,NaN,1993-03-14,1993-03
158,3669816,5270,credit,Missing,259.3,49085.0,interest credited,Missing,NaN,1993-03-31,1993-03


In [23]:
#collection from another bank aparece no máx uma vez por mês
gb_rev=trans_dev.loc[trans_dev.operation=='collection from another bank'].groupby(['account_id','generation']).agg({'operation': 'count', 'amount': 'sum'}).reset_index()
gb_rev.loc[gb_rev.operation>1]

,account_id,generation,operation,amount


In [24]:
#opeartion=Missing <=> k_symbol=interest credited
trans_dev.loc[(trans_dev.operation=='Missing') & (trans_dev.k_symbol!='interest credited')]

,trans_id,account_id,type,operation,amount,balance,k_symbol,bank,account,trans_date,generation


In [25]:
#opeartion=Missing <=> k_symbol=interest credited
trans_dev.loc[(trans_dev.operation!='Missing') & (trans_dev.k_symbol=='interest credited')]

,trans_id,account_id,type,operation,amount,balance,k_symbol,bank,account,trans_date,generation


## Informações de Conta

In [26]:
account_d=account.merge(disp,left_on='account_id',right_on='account_id',how='inner')
#4500 linhas para 5369 (ok)

In [27]:
account_dc=account_d.merge(client,left_on='client_id',right_on='client_id',how='inner',suffixes=('_ac', '_cli'))
#mantem 5369
#dados com incongruencias no district (percebido que um é da filial e outro do cliente)

In [28]:
account_dc.loc[account_dc.district_id_ac!=account_dc.district_id_cli,'change_district']=1
account_dc.loc[account_dc.district_id_ac==account_dc.district_id_cli,'change_district']=0

In [29]:
#um exemplo da diferença
account_d.loc[account_d.client_id==692]

,account_id,district_id,frequency,ac_date,disp_id,client_id,type
0,576,55,monthly issuance,1993-01-01,692,692,OWNER


In [30]:
client.loc[client.client_id==692]

,client_id,district_id,sex,birth_date
660,692,74,F,1936-01-11


In [31]:
account_dc.columns

Index(['account_id', 'district_id_ac', 'frequency', 'ac_date', 'disp_id',
       'client_id', 'type', 'district_id_cli', 'sex', 'birth_date',
       'change_district'],
      dtype='object')

In [32]:
#organizar colunas
account_dc=account_dc[['account_id', 'district_id_ac', 'district_id_cli', 'disp_id', 'client_id', 
                       'ac_date', 'birth_date',
                        'frequency','type', 'sex', 'change_district']]

In [33]:
account_dcd=account_dc.merge(district, left_on='district_id_cli', right_on='code', how='inner')
account_dcd.drop(['code', 'client_id'],axis=1,inplace=True)
#mantem 5369

In [34]:
account_dcd.columns

Index(['account_id', 'district_id_ac', 'district_id_cli', 'disp_id', 'ac_date',
       'birth_date', 'frequency', 'type', 'sex', 'change_district', 'name',
       'region', 'no_inhabitants', 'no_municipalities_with_inhabitants_499',
       'no_municipalities_inhabitants_500_1999',
       'no_municipalities_with_inhabitants_2000_9999',
       'no_municipalities_with_inhabitants_10000', 'no_cities',
       'ratio_urban_inhabitants', 'average_salary', 'unemploymant_rate_95',
       'unemploymant_rate_96', 'no_enterpreneurs_per_1000_inhabitants',
       'no_commited_crimes_95', 'no_commited_crimes_96'],
      dtype='object')

## Créditos com cartão

In [35]:
card_ac=card_dev.merge(account_dcd,left_on='disp_id',right_on='disp_id',how='inner',suffixes=('_card', '_disp'))

card_ac.drop(['card_id'],axis=1,inplace=True)

In [36]:
#organizar a base de dados
card_ac=card_ac[['disp_id', 'account_id', 
       'issued_date', 'ac_date', 'birth_date',
       'type_card', 'frequency', 'type_disp', 'sex', 'change_district',
       'name', 'region', 'no_inhabitants',
       'no_municipalities_with_inhabitants_499',
       'no_municipalities_inhabitants_500_1999',
       'no_municipalities_with_inhabitants_2000_9999',
       'no_municipalities_with_inhabitants_10000', 'no_cities',
       'ratio_urban_inhabitants', 'average_salary', 'unemploymant_rate_95',
       'unemploymant_rate_96', 'no_enterpreneurs_per_1000_inhabitants',
       'no_commited_crimes_95', 'no_commited_crimes_96']]

#rename colunamas
card_ac.rename(columns={'issued_date':'product_date', 'type_card':'card_type',}, inplace=True)

## Créditos clássicos

In [37]:
loan_dev.head()

,loan_id,account_id,amount,duration,payments,status,loan_date
0,5314,1787,96396,12,8033,1,1993-07-05
1,5316,1801,165960,36,4610,0,1993-07-11
2,6863,9188,127080,60,2118,0,1993-07-28
3,5325,1843,105804,36,2939,0,1993-08-03
4,7240,11013,274740,60,4579,0,1993-09-06


In [38]:
loan_ac=loan_dev.merge(account_dcd)
#passa de 328 linhas para 403.
# ---> Podemos considerar 2º tit aqui?????

#loan_ac=loan_dev.merge(account_dcd.loc[account_dcd.type=='OWNER']) #este não considera 2º tit

loan_ac.drop(['loan_id'],axis=1,inplace=True)

In [39]:
#organizar a base de dados
loan_ac=loan_ac[['status', 'account_id', 'disp_id',
       'loan_date', 'ac_date', 'birth_date', 
       'amount', 'duration', 'payments', 'frequency', 'type',
       'sex', 'change_district', 'name', 'region', 'no_inhabitants',
       'no_municipalities_with_inhabitants_499',
       'no_municipalities_inhabitants_500_1999',
       'no_municipalities_with_inhabitants_2000_9999',
       'no_municipalities_with_inhabitants_10000', 'no_cities',
       'ratio_urban_inhabitants', 'average_salary', 'unemploymant_rate_95',
       'unemploymant_rate_96', 'no_enterpreneurs_per_1000_inhabitants',
       'no_commited_crimes_95', 'no_commited_crimes_96']]

#rename colunamas
loan_ac.rename(columns={'loan_date':'product_date', 'type':'type_disp',}, inplace=True)

In [40]:
#existem clientes com cartão na tabela loan!!
loan_ac.loc[loan_ac.disp_id.isin(card_dev.disp_id)].shape

(11, 28)

In [41]:
#creditos = pd.concat([card_ac,loan_ac])